In [1]:
from omop_constructs.alchemy.adapters import map_lookup_view
from omop_constructs.alchemy.adapters import make_and_map_lookup_view, make_source_lookup
from omop_alchemy.cdm.model.clinical import Condition_Occurrence, Measurement
import sqlalchemy as sa
from orm_loader.helpers import Base

In [4]:
# this factory form can be used when you want to create 
# a lookup to your source data to match directly - 
# in this example, we want to match the condition_status_source_value 
# to the med_id in our source data, so we can use that to link back to source
# while accommodating type directly for convenience

# it also tracks where you've stored the source ID in your schema...which exposes some non-standard usage...

condition_source_lookup = make_source_lookup(
    model=Condition_Occurrence,
    source_column=Condition_Occurrence.condition_status_source_value,
    source_label="med_id",
    cast_to=sa.Integer(),
    name="condition_source_lookup",  # optional but nice
)

Condition_Source = map_lookup_view(
    base=Base,
    subquery=condition_source_lookup,
    name="Condition_Source",
    pk_cols=["condition_occurrence_id"],
)

In [5]:
list(Condition_Source.__table__.columns)

[Column('person_id', Integer(), ForeignKey('person.person_id'), table=<condition_source_lookup>, nullable=False),
 Column('condition_occurrence_id', Integer(), table=<condition_source_lookup>, primary_key=True, nullable=False),
 Column('condition_concept_id', Integer(), ForeignKey('concept.concept_id'), table=<condition_source_lookup>, nullable=False),
 Column('condition_start_date', Date(), table=<condition_source_lookup>, nullable=False),
 Column('condition_end_date', Date(), table=<condition_source_lookup>),
 <sqlalchemy.sql.elements.ColumnClause at 0x12528aad0; med_id>]

In [2]:
# or in a single step...

Measurement_Source = make_and_map_lookup_view(
    base=Base,
    name="Measurement_Source",
    model=Measurement,
    source_column=Measurement.measurement_source_value,
    source_label="obd_id",
    cast_to=sa.String(),
    pk_cols=["measurement_id"],
)

In [6]:
list(Measurement_Source.__table__.columns)

[Column('person_id', Integer(), ForeignKey('person.person_id'), table=<measurement_source>, nullable=False),
 Column('measurement_id', Integer(), table=<measurement_source>, primary_key=True, nullable=False),
 <sqlalchemy.sql.elements.ColumnClause at 0x124f546e0; obd_id>]